# Import libraries

In [2]:
from text_matcher.matcher import Text, Matcher
import json
import pandas as pd
from IPython.display import clear_output
%matplotlib inline
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = [16, 6]
#pd.set_option('display.max_colwidth', None)

# Load in our Data Files

In [6]:
# Load Middlemarch .txt file 
# (Note: must have 'middlemarch.txt' in this directory)
with open('../algorithm-testing/middlemarch.txt') as f: 
    rawMM = f.read()

mm = Text(rawMM, 'Middlemarch')

# Load in the JSON file with our JSTOR articles and data from TextMatcher
# (Note: must have the file 'default.json' in the same directory as this notebook)
df = pd.read_json('../algorithm-testing/jstor-middlemarch-articles.json')

# Importing the spell-check package

## Input an article_id to run the spell-check on

Languages tested:
* English - ‘en’
* Spanish - ‘es’
* French - ‘fr’
* German - ‘de’

In [21]:
from spellchecker import SpellChecker

# ‼️ 🛑 Make sure to change the variable below to the correct article id 🛑  ‼️
article_id  = 'http://www.jstor.org/stable/439034' # CHANGE THIS to article id

spell_check_language = 'en'

spell = SpellChecker(language = spell_check_language)

# Use article_id to get the index of the article in our DataFrame
article_index = df[df['id'] == article_id].index[0]
article_text = df['fullText'].loc[article_index]
article_title = df['title'].loc[article_index]

# Assign the full text of this article to a variable called `cleaned_article_text`, with text-matcher's Text function
cleaned_article_text = Text(article_text, article_title)

word_list = ((" ").join(article_text)).split(" ")

# find those words that may be misspelled
incorrect = []
for lang in ['en','fr','es','de']:
    spell = SpellChecker(language = lang)
    misspelled = spell.unknown(word_list)
    incorrect.append(len(misspelled))

incorrect_percentage = float(min(incorrect))/len(word_list)

print("Percent incorrect:", incorrect_percentage)


Percent incorrect: 0.16329941860465116
